<a href="https://colab.research.google.com/github/salarbalou/Data_Analysis_Projects/blob/main/PID_Tank_Liquid_Level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Single tank PID level control

The liquid level in a tank with an inflow and gravity drainage is controlled to a desired setpoint $h_{sp}$ by manipulating by an adjustable flow $q_{mv}$ with a proportional - integral - derivative (PID) controller.  A negative $q_{mv}$ indicates that liquid is pumped out of the tank.  The controller parameters are $k_c$, $\tau_i$, and $\tau_d$.

$$A \dfrac{dh}{dt} = q_{in} - C_v \sqrt h + q_{mv} $$

$$ q_{mv} = q_{mv,0} + k_c \left(
  \epsilon + \dfrac{1}{\tau_i} \int_0^t \epsilon \ d \tau  + \dfrac{1}{\tau_d} \dfrac{d\epsilon}{dt}\right)$$ \\

$$h_0 = \left( \dfrac{q_{mv,0} + q_{in,0}}{C_v} \right)^2$$


$$  \epsilon = h - h_{sp}$$


<br><br> 

$$A \dfrac{dh}{dt} = 
  q_{in} - C_v \sqrt h + k_c \left(
  \epsilon + \dfrac{1}{\tau_i} \epsilon_{int}  + \dfrac{1}{\tau_d} \left(\dfrac{dh}{dt} - \dfrac{dh_{sp}}{dt} \right) \right)$$
$$\dfrac{dh}{dt} = \left(\dfrac{1}{A - \frac{k_c}{\tau_d}}\right)
  \left(
    q_{in} - C_v \sqrt h + k_c \left(
  \epsilon + \dfrac{1}{\tau_i} \epsilon_{int}  - \dfrac{1}{\tau_d} \dfrac{dh_{sp}}{dt} \right)
  \right)$$
$$ \dfrac{d \epsilon_{int}}{dt} = \epsilon$$

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

In [ ]:
def dh(t, v, hsp, dhsp, qin, qmv0, A, cv, kc, taui, taud):
    h, errint = v
    err = v[0] - hsp
    return [(qin(t) - cv*v[0]**0.5 + qmv0 + kc*(err + v[1]/taui - dhsp/taud))/(A-kc/taud),err]

In [ ]:
tend = 30
A = 1
cv = 1
kc = -1
taui = 0.2
taud = 1
hsp = 1.5
dhsp = 0

errint0 = 0

def qin(t):
  return 0.2

qmv0 = 0.2
h0 = ((qin(0)+qmv0)/cv)**2
hsp=h0*1.1
res = solve_ivp(lambda t,v: dh(t,v, hsp, dhsp, qin, qmv0, A, cv, kc, taui, taud), (0,tend), [h0, errint0], 
                dense_output=True, method='Radau')

t_plot = np.linspace(0,tend, 200)
h, err_int = res.sol(t_plot)
err = h-hsp
qmv = qmv0+kc*(err + err_int/taui - dhsp/taud)
fig=make_subplots(rows=1,cols=2)
fig.add_scatter(x=t_plot,y=h, row=1,col=1,name='height')
fig.add_scatter(x=t_plot,y=qmv, row=1,col=2,name='qm')

### 2 Interacting Tanks In Series
Flow between tanks set by difference in height of fluid in the two tanks.  Setpoint on height of fluid in second tank.

$$A_1 \dfrac{dh_1}{dt} = 
  q_{in} - C_{v1} \sqrt{h_1-h_2} + q_{mv}
$$

$$q_{mv} = q_{mv0} + k_c \left(
  \epsilon + \dfrac{1}{\tau_i} \epsilon_{int}  + \dfrac{1}{\tau_d} \left(\dfrac{dh_2}{dt} - \dfrac{dh_{sp}}{dt} \right) \right)$$

$$A_2 \dfrac{dh_2}{dt} = 
  C_{v1} \sqrt{h_1-h_2} - C_{v2} \sqrt{h_2}
$$

$$ \epsilon = h_2 - h_{sp}$$

$$ \dfrac{d \epsilon_{int}}{dt} = \epsilon$$

$$ h_{2,0} = \left( \dfrac{q_{in,0} + q_{mv,0}}{C_{v2}} 
\right)^2$$

$$h_{1,0} = h_{2,0} \left( \left( \dfrac{C_{v,2}}{C_{v,1}} \right)^2  +1 \right)$$

In [ ]:
def d2tanks(t, v, hsp, dhsp, qin, qmv0, A1, A2, cv1, cv2, kc, taui, taud):
    h1,h2,err_int = v

    err = h2 - hsp
    q12 = cv1 * (h1-h2)**0.5
    dh2 = (q12 - cv2*h2**0.5)/A2
    dh1 = (qin(t) - q12 + qmv0+ kc*(err + err_int/taui + (dh2-dhsp)/taud))/A1

    return [dh1, dh2, err]

tend = 10
A1 = 1
A2 = 1
cv1 = 1
cv2 = 1
kc = -20
taui = 0.3
taud = 2
hsp = 0.5
dhsp = 0

qmv0 = 0.2
errint0 = 0

def qin(t):
  return 0.2

h2_0 = ((qin(0)+qmv0)/cv2)**2 
h1_0 = h2_0 * ((cv2/cv1)**2 +1 )

res = solve_ivp(lambda t,v: d2tanks(t,v, hsp, dhsp, qin, qmv0, A1, A2, cv1, cv2, kc, taui, taud), 
                (0,tend), [h1_0, h2_0, errint0], dense_output=True, method='Radau')
h1,h2,err_int = res.sol(t_plot)
err = h2 - hsp
q12 = cv1 * (h1-h2)**0.5
dh2 = (q12 - cv2*h2**0.5)/A2
qmv = qmv0+kc*(err + err_int/taui + (dh2-dhsp)/taud)

t_plot = np.linspace(0,tend, 200)

fig=make_subplots(rows=1,cols=3)
fig.add_scatter(x=t_plot,y=h1, row=1,col=1,name='h1')
fig.add_scatter(x=t_plot,y=h2, row=1,col=2,name='h2')
fig.add_scatter(x=t_plot,y=qmv, row=1,col=3,name='qm')

### Cascade Control
The control action on the manipulated flow into Tank 1 takes time to propagate to the measured height of Tank 2.  A significant improvement in controller performance is achievable using a master-slave control strategy.  In this so-called _cascade control_ strategy, the liquid height in Tank 1 is controlled by a slave P-only controller which takes as setpoint the response from a master PI-controller on Tank 2.  

Tank 1:
$$A_1 \dfrac{dh_1}{dt} = 
  q_{in} - C_{v1} \sqrt{h_1-h_2} + q_{mv}
$$

$$q_{mv} = q_{mv,0} + k_{1c} \left(h_1 - h_{1sp} \right) $$

$$A_2 \dfrac{dh_2}{dt} = 
  C_{v1} \sqrt{h_1-h_2} - C_{v2} \sqrt{h_2}
$$

$$h_{1sp} = h_{1sp,0} + k_{2,c} \left(\epsilon + \dfrac{1}{\tau_{2,i}} \epsilon _{int} \right)$$

$$ \epsilon = h_2 - h_{2,sp}$$

$$ \dfrac{d \epsilon_{int}}{dt} = \epsilon$$

$$ h_{2,0} = \left( \dfrac{q_{in,0} + q_{mv,0}}{C_{v2}} 
\right)^2$$

$$h_{1,0} = h_{2,0} \left( \left( \dfrac{C_{v,2}}{C_{v,1}} \right)^2  +1 \right)$$

$$h_{1sp,0} = h_{1,0}$$

In [ ]:
def d2tanks_cascade_control(t, v, hsp, dhsp, qin, qmv0, h1sp_0, A1, A2, cv1, cv2, k1c, k2c, tau2i):
    h1,h2,err_int = v
    err = h2 - hsp
    h1sp = h1sp_0 + k2c*(err + err_int/tau2i)
    q12 = cv1 * (h1-h2)**0.5
    dh2 = (q12 - cv2*h2**0.5)/A2
    qmv = qmv0+ k1c*(h1-h1sp)
    dh1 = (qin(t) - q12 + qmv)/A1

    return [dh1, dh2, err]

tend = 3
A1 = 1
A2 = 1
cv1 = 1
cv2 = 1
k1c = -20
k2c = -10
tau2i = 1
hsp = 0.5
dhsp = 0

errint0 = 0

def qin(t):
  return 0.3+0.2*np.sin(5*t)

qmv0 = 0.2
errint0 = 0

h2_0 = ((qin(0)+qmv0)/cv2)**2 
h1_0 = h2_0 * ((cv2/cv1)**2 +1 )
h1sp_0 = h1_0

res = solve_ivp(lambda t,v: d2tanks_cascade_control(t,v, hsp, dhsp, qin, qmv0, h1sp_0, A1, A2, cv1, cv2, k1c, k2c, tau2i ), 
                (0,tend), [h1_0, h2_0, errint0], dense_output=True, method='Radau')

t_plot = np.linspace(0,tend, 200)
h1, h2, err_int = res.sol(t_plot)
h2 = res.sol(t_plot)[1]

err = h2 - hsp
q12 = cv1 * (h1-h2)**0.5
dh2 = (q12 - cv2*h2**0.5)/A2
h1sp = h1sp_0 + k2c*(err + err_int/tau2i)
qmv = qmv0+ k1c*(h1-h1sp)

fig=make_subplots(rows=1,cols=3)
fig.add_scatter(x=t_plot,y=h1, row=1,col=1,name='h1')
fig.add_scatter(x=t_plot,y=h2, row=1,col=2,name='h2')
fig.add_scatter(x=t_plot,y=qmv, row=1,col=3,name='qm')

### Manipulated Flow Positive Only - Cannot Pump Out

What happens if liquid cannot be pumped out of Tank 1?  In this case, the liquid height in Tank 1 cannot be aggressively increased initially to fill up Tank 2 quickly.  With no means to pump out liquid from Tank 1, this would lead to a significant period of overshoot.  In a straight-line car race where one must pass the finish line at no more than 5 mph, a professional driver cannot do much better than an amateur on a car that has no brakes!

In [ ]:
def d2tanks_cascade_control(t, v, hsp, dhsp, qin, qmv0, h1sp_0, A1, A2, cv1, cv2, k1c, k2c, tau2i):
    h1,h2,err_int = v
    err = h2 - hsp
    h1sp = h1sp_0 + k2c*(err + err_int/tau2i)
    q12 = cv1 * (h1-h2)**0.5
    dh2 = (q12 - cv2*h2**0.5)/A2
    qmv = qmv0+ k1c*(h1-h1sp)
    qmv = max(qmv, 0)
    dh1 = (qin(t) - q12 + qmv)/A1

    return [dh1, dh2, err]

tend = 5
A1 = 1
A2 = 1
cv1 = 1
cv2 = 1
k1c = -20
k2c = -4
tau2i = 0.4
hsp = 0.5
dhsp = 0

errint0 = 0

def qin(t):
  return 0.3+0.2*np.sin(5*t)

qmv0 = 0.2
errint0 = 0

h2_0 = ((qin(0)+qmv0)/cv2)**2 
h1_0 = h2_0 * ((cv2/cv1)**2 +1 )
h1sp_0 = h1_0

res = solve_ivp(lambda t,v: d2tanks_cascade_control(t,v, hsp, dhsp, qin, qmv0, h1sp_0, A1, A2, cv1, cv2, k1c, k2c, tau2i ), 
                (0,tend), [h1_0, h2_0, errint0], dense_output=True, method='Radau')

t_plot = np.linspace(0,tend, 200)
h1, h2, err_int = res.sol(t_plot)
h2 = res.sol(t_plot)[1]

err = h2 - hsp
q12 = cv1 * (h1-h2)**0.5
dh2 = (q12 - cv2*h2**0.5)/A2
h1sp = h1sp_0 + k2c*(err + err_int/tau2i)
qmv = qmv0+ k1c*(h1-h1sp)
qmv = np.where(qmv<0, 0, qmv)

fig=make_subplots(rows=1,cols=3)
fig.add_scatter(x=t_plot,y=h1, row=1,col=1,name='h1')
fig.add_scatter(x=t_plot,y=h2, row=1,col=2,name='h2')
fig.add_scatter(x=t_plot,y=qmv, row=1,col=3,name='qm')